In [3]:
from tensorflow import keras 
import cv2
import numpy as np
model = keras.models.load_model("mask_model95.5.h5")

In [ ]:
def preprocess_image(frame):
    frame = cv2.resize(frame, (224, 224))  # Resize to match model input
    frame = frame.astype('float32') / 255.0  # Normalize pixel values
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension (1, 224, 224, 3)
    return frame
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        break

    # # Preprocess the frame for prediction
    # processed_frame = preprocess_image(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    # Detects faces of different sizes in the input image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        # To draw a rectangle in a face 
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2) 
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

    # # Make prediction
    roi_color_process = preprocess_image(frame)
    prediction = model.predict(roi_color_process)
    predicted_class = 'mask on' if prediction[0] > 0.7 else 'Please wear mask'  # Adjust for your classes

    # # Display the prediction on the video feed
    # if predicted_class == 0:

    #     cv2.F
        
    cv2.putText(frame, f"Prediction: {predicted_class}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    cv2.imshow('Live Camera Feed', frame)  # Show the frame with prediction

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



In [7]:
cap.release()
cv2.destroyAllWindows()